# Analisar discursos

O objetivo desse notebook é analisar discursos disponíveis no diretório _data. Para preparar uma nova base, execute o notebook 01_preparar_base_discursos.ipynb

## Carregar dados

In [ ]:
%pip install datasets

In [ ]:
import pandas as pd
from datasets import load_dataset
#from pathlib import Path

#DATA = Path("_data/discursos_2019-02-01_2023-01-31.parquet")
#df = pd.read_parquet(DATA)

data_files = {"train": "data/full/discursos_2019-02-01_2023-01-31.parquet"}
dataset = load_dataset("fabriciosantana/discursos-senado-legislatura-56", data_files=data_files)    

In [ ]:
df = pd.DataFrame(dataset["train"])
#df = dataset["train"].to_pandas()

df.head()

In [ ]:
df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
df = df[df["ok"].eq(True) & df["TextoDiscursoIntegral"].notna() & df["TextoDiscursoIntegral"].str.len().gt(0)]
df["ano"] = df["Data"].dt.year
df["mes"] = df["Data"].dt.to_period("M").astype(str)  # ex.: '2019-03'

df.head()

## Mapa de temas (tópicos)

In [ ]:
import re
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

def normalize(s):
    s = unidecode((s or "").lower())
    s = re.sub(r"[^a-z0-9áéíóúãõç\s]", " ", s)  # simples; ajuste se quiser
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["texto_clean"] = df["TextoDiscursoIntegral"].apply(normalize)

# embeddings (semântica)
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
emb = model.encode(df["texto_clean"].tolist(), batch_size=64, show_progress_bar=True, normalize_embeddings=True)

# clusterização
k = 20  # ajuste depois de olhar coerência
km = KMeans(n_clusters=k, n_init=10, random_state=42)
df["topico"] = km.fit_predict(emb)

# palavras-chave por tópico via TF-IDF médio (c-TF-IDF simplificado)
vec = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.8)
X = vec.fit_transform(df["texto_clean"])
vocab = np.array(vec.get_feature_names_out())

def top_terms_for_topic(t, n=12):
    idx = np.where(df["topico"].values == t)[0]
    if len(idx)==0: return []
    mean_tfidf = X[idx].mean(axis=0).A1
    return vocab[mean_tfidf.argsort()[::-1][:n]].tolist()

termos = {t: top_terms_for_topic(t) for t in sorted(df["topico"].unique())}
pd.DataFrame({"topico": list(termos.keys()), "termos": list(termos.values())}).head()


### Tendências (mês × tópico)

In [ ]:
trend = (df.groupby(["mes","topico"])
           .size()
           .reset_index(name="qtd"))
# se quiser por partido: .groupby(["mes","topico","Partido"]).size()...
trend.head()

### Tom/sentimento (baseline) e variações por grupo

In [ ]:
pos = {"importante","saude","melhoria","progresso","direito","justica","educacao"}
neg = {"crise","problema","corrupcao","retrocesso","corte","injustica","deficit"}

def lex_sent(s):
    toks = normalize(s).split()
    p = sum(w in pos for w in toks); n = sum(w in neg for w in toks)
    return (p - n) / max(1, len(toks))

df["sent_lex"] = df["TextoDiscursoIntegral"].apply(lex_sent)
sent_partido = df.groupby("Partido")["sent_lex"].mean().sort_values(ascending=False)


In [ ]:
top_oradores = (df.groupby(["topico","NomeAutor"])
                  .size()
                  .reset_index(name="falas")
                  .sort_values(["topico","falas"], ascending=[True,False]))
top_partidos = (df.groupby(["topico","Partido"])
                  .size()
                  .reset_index(name="falas")
                  .sort_values(["topico","falas"], ascending=[True,False]))


In [ ]:
!pip install spacy
import spacy

nlp = spacy.load("pt_core_news_lg")  # python -m spacy download pt_core_news_lg

def ents(text):
    doc = nlp(text[:20000])  # limite por desempenho
    return [(e.text, e.label_) for e in doc.ents]

sample = df.sample(500, random_state=42).copy()  # rode em lote
sample["ents"] = sample["TextoDiscursoIntegral"].apply(ents)

# co-ocorrência simples (pessoas dentro do mesmo discurso)
from collections import Counter
pairs = Counter()
for lst in sample["ents"]:
    ps = sorted({e for e,l in lst if l in ("PER","ORG")})
    for i in range(len(ps)):
        for j in range(i+1, len(ps)):
            pairs[(ps[i], ps[j])] += 1

pd.DataFrame([{"a":a,"b":b,"peso":w} for (a,b),w in pairs.items()]).sort_values("peso", ascending=False).head(20)


In [ ]:
df = pd.read_csv("_data/discursos_2019-01-01_2023-01-31.csv", sep=";")
df.head()